In [9]:
import os
%run "{os.path.dirname(os.getcwd())}\general_functions\generalFunctions.ipynb" #container

%run "{os.path.dirname(os.getcwd())}\general_functions\Extracting Data Functions.ipynb" # container

In [10]:
import time
import adodbapi
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

In [11]:
start_time = time.time()

In [12]:
filename = 'parameters.xlsx'

# Get the current working directory
current_dir = os.path.dirname(os.getcwd())

# Construct the full path to the file
f_path = os.path.join(current_dir, filename)
print(f_path)
#xls = pd.ExcelFile(f_path)
parm = pd.read_excel(f_path, sheet_name='Landscape')
fields = dict(zip(parm['Field'],parm['Value']))

c:\Users\Ali Salem\Desktop\App_Update\parameters.xlsx


In [13]:
server = "powerbi://api.powerbi.com/v1.0/myorg/"+ fields['server']
dataset_name = fields['f_name']
f_name = os.getcwd()+"/"+fields['f_name']+".xlsx"

client_manuf = list(set(fields['client_manuf'].split(','))-set(['']))
client_brands = list(set(fields['client_brands'].split(','))-set(['']))

decimals = fields['decimals']
sign = fields['sign']
currency = fields['currency']
currency = ' '+ currency if sign.lower() == 'after' else  currency + ' ' 

categories = list(set(fields['categories'].split(','))-set(['']))
sectors=list(set(fields['sectors'].split(','))-set(['']))
segments=list(set(fields['segments'].split(','))-set(['']))
subsegments=list(set(fields['subsegments'].split(','))-set(['']))
subcategories=list(set(fields['subcategories'].split(','))-set(['']))

national=fields['national']
customareas=fields['customareas']
areas = list(set(fields['areas'].split(','))-set(['']))+[customareas]

regions_RET = list(set(fields['regions_RET'].split(','))-set(['']))
channels_RET = list(set(fields['channels_RET'].split(','))-set(['']))
market_RET=list(set(fields['market_RET'].split(','))-set(['']))

regions_CHAN=list(set(fields['regions_CHAN'].split(','))-set(['']))
channels_CHAN=list(set(fields['channels_CHAN'].split(','))-set(['']))
market_CHAN=list(set(fields['market_CHAN'].split(','))-set(['']))

regions_CUST=list(set(fields['regions_CUST'].split(','))-set(['']))
channels_CUST=list(set(fields['channels_CUST'].split(','))-set(['']))
market_CUST=list(set(fields['market_CUST'].split(','))-set(['']))

data_source=fields['data_source']
years=list(set(fields['years'].split(','))-set(['']))
years = {int(y) for y in fields['years'].split(',') if y}

end_date=fields['end_date']

ManufOrTopC = fields['ManufOrTopC']
BrandOrTopB = fields['BrandOrTopB']

National=["NATIONAL"]if national else[]



In [14]:
years

{2023, 2024, 2025}

In [15]:
regions = regions_RET + regions_CHAN + regions_CUST
channels = channels_RET + channels_CHAN + channels_CUST
markets = market_RET + market_CHAN + market_CUST


entity_hierarchy = [
    ("Area",National),
    ("Region", regions),
    ("Channel", channels),
    ("Market", markets)
]
hierarchy_levels = [
    ("Category", categories),
    ("Sector", sectors),
    ("Segment", segments),
    ("SubSegment", subsegments),
    ("SubCategory", subcategories)
]

In [16]:
channels

["Sam's Corp", 'Walmart', "Bj's Corp"]

In [17]:
conn_str = f"Provider=MSOLAP.8;Data Source={server};Initial Catalog={dataset_name};"
path=os.path.join(os.getcwd(),"Landscape Datasets Test")

In [18]:
# client_manuf = ["Pbg"]
# client_brands = ["Schick", "Cremo", "Equate"]
 
# decimals = 2
# sign = "Before"
# currency = '$'
# currency = ' '+ currency if sign.lower() == 'after' else  currency + ' '
 

# categories = ["Manual Shave Men"]
# sectors = ["System", "Disposables"]
# segments = ["Razors", "Refills", "Disposables"]
# subsegments= ["2 Blade", "3 Blade",	"4 Blade", "5 Blade", "6 Blade", "Rem Blades"]
# subcategories= ["Dry Skin",	"Normal Skin", "Sensitive Skin", "Rem Types"]

# national = True
# customareas= ""
# areas = ["NATIONAL", "RETAILER", "CHANNEL"]

# regions_RET  = []
# channels_RET = ["Bj's Corp", "Sam's Corp", "Walmart"]
# market_RET = []

# regions_CHAN = ["Conv",	"Drug",	"Food"]
# channels_CHAN = []
# market_CHAN = []

# regions_CUST = []
# channels_CUST = []
# market_CUST = []
 

# data_source = "DATA SOURCE: Trade Panel/Retailer Data | Ending March  2025" 

# ManufOrTopC ="Top Companies"
# BrandOrTopB = "SubBrand"

# years = {2023,2024,2025}

# National=["NATIONAL"]if national else[]
# regions = regions_RET + regions_CHAN + regions_CUST
# channels = channels_RET + channels_CHAN + channels_CUST
# markets = market_RET + market_CHAN + market_CUST


# entity_hierarchy = [
#     ("Area",National),
#     ("Region", regions),
#     ("Channel", channels),
#     ("Market", markets)
# ]
# hierarchy_levels = [
#     ("Category", categories),
#     ("Sector", sectors),
#     ("Segment", segments),
#     ("SubSegment", subsegments),
#     ("SubCategory", subcategories)
# ]
# server = "powerbi://api.powerbi.com/v1.0/myorg/Edgewell"
# dataset_name = "Edgewell US Male Dataset"
# conn_str = f"Provider=MSOLAP.8;Data Source={server};Initial Catalog={dataset_name};"

# print(conn_str)
# path=os.path.join(os.getcwd(),"Landscape Datasets NewEX")


## Hierarchy Dataframes

In [19]:
def execute_dax_query(entity_name, area, hierby, filter_p12m, include_year=False):
    outputdic = {}
    
    time_filter = """
        FILTER(
            VALUES('Time Logic'[Time Period]), 
            'Time Logic'[Time Period] = "P12M"
        )
    """ if filter_p12m else ""
    key =  f"{categories[0]} | {entity_name}"
    filters = ", ".join(filter(None, [f'Products[Category] = "{categories[0]}"', time_filter]))

    columns = [
        "Volume Sales", "Value Sales", "Value Share", "Volume Share", "Value Sales IYA",
        "Av Price/KG", "WoB %", "Relative Price", "Growth Contribution",
        "Volume Sales IYA", "IYA Price/KG"
    ]
    
    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)

    dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(Products, Products[{hierby}]),
                {column_exprs}
            ),
            {filters},
            TREATAS({{"{entity_name}"}}, Market[{area}]),
            FILTER('Scope', 'Scope'[Scope] = "Category")
        )   
    """

    grandtotal_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                Values(Products[Category]),
                {column_exprs}
            ),
            {filters},
            TREATAS({{"{entity_name}"}}, Market[{area}]),
            FILTER('Scope', 'Scope'[Scope] = "{hierby}")
        )   
    """
    try:
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(dax_query)
            columns = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()
            

        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(grandtotal_query)
            grandtotal_columns = [desc[0] for desc in cursor.description]
            grandtotal_data = cursor.fetchall()
            
            df = pd.DataFrame(data, columns=columns)
            df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)         
            df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]  # Remove zero rows

            grand_tot = pd.DataFrame(grandtotal_data, columns=grandtotal_columns)
            grand_tot.columns = grand_tot.columns.str.replace(r'.*\[|\]', '', regex=True)
            grand_tot = grand_tot.loc[~(grand_tot.select_dtypes(include='number') == 0).all(axis=1)]  # Remove zero rows

            grand_tot[df.columns[0]] = 'Grand Total'

            # Reorder columns if necessary
            grand_tot = grand_tot[df.columns]

            # Concatenate the two
            df = pd.concat([df, grand_tot], ignore_index=True)
            outputdic[key] = df  

            
            print(f"Query executed successfully for {entity_name}.")
    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_name} in {area}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_name} in {area}: {e}")

    return outputdic



def process_dax_queries(entity_hierarchy, hierarchy_levels, time_filter):
    with ThreadPoolExecutor(max_workers=8) as executor:
        dfs_results = {} 
        futures = {}
        ordered_keys=[]
        for hierby, hier_values in hierarchy_levels:
            if hierby == "Category":
                continue
            for area, entity_list in entity_hierarchy:
                for entity in entity_list:
                    include_year = not time_filter 
                    key =  f"{categories[0]} | {entity}"
                    ordered_keys.append(key)
                    future = executor.submit(execute_dax_query, entity, area, hierby, time_filter, include_year=include_year)
                    futures[future] = key
       
            temp_results = {}
            for future in as_completed(futures):
                result = future.result()
                temp_results.update(result)

            # Insert results in original order
            for key in ordered_keys:
                if key in temp_results:
                    dfs_results[key] = temp_results[key]
          
            filename =  f"{hierby}_P12M_dfs.pkl"
            
            output_file = f"{path}\\{filename}"
            with open(output_file, "wb") as f:
                pd.to_pickle(dfs_results, f)
            
            print(f"All DataFrames saved to {output_file}.")

process_dax_queries(entity_hierarchy,hierarchy_levels, time_filter=True)   # sectors_12_dfs.pkl



Query executed successfully for Sam's Corp.
Query executed successfully for Food.
Query executed successfully for NATIONAL.
Query executed successfully for Conv.
Query executed successfully for Walmart.
Query executed successfully for Bj's Corp.
Query executed successfully for Drug.
All DataFrames saved to c:\Users\Ali Salem\Desktop\App_Update\Landscape\Landscape Datasets Test\Sector_P12M_dfs.pkl.
Query executed successfully for Walmart.
Query executed successfully for Conv.
Query executed successfully for Sam's Corp.
Query executed successfully for Bj's Corp.
Query executed successfully for Food.
Query executed successfully for Drug.
Query executed successfully for NATIONAL.
All DataFrames saved to c:\Users\Ali Salem\Desktop\App_Update\Landscape\Landscape Datasets Test\Segment_P12M_dfs.pkl.
Query executed successfully for Bj's Corp.
Query executed successfully for Food.
Query executed successfully for Sam's Corp.
Query executed successfully for Conv.
Query executed successfully for Dr

In [20]:
def execute_dax_query(entity_name, area, hierby, filter_p12m, include_year=False):
    outputdic = {}
    outputtotal = {}
 
    key = f"{categories[0]} | {entity_name}"
 
    # Optional time filter
    time_filter = """
        FILTER(
            VALUES('Time Logic'[Time Period]),
            'Time Logic'[Time Period] = "P12M"
        )
    """ if filter_p12m else ""
 
    # Combine filters
    filters = ", ".join(filter(None, [
        f'Products[Category] = "{categories[0]}"',
        time_filter
    ]))
 
    # Define all required columns
    columns = [
        "Volume Sales", "Value Sales", "Value Share", "Volume Share", "Value Sales IYA",
        "Av Price/KG", "WoB %", "Relative Price", "Growth Contribution",
        "Volume Sales IYA", "IYA Price/KG"
    ]
    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)
 
    # Main DAX query
    summarize_expr = (
        f"CROSSJOIN(SUMMARIZE(Products, Products[{hierby}]), VALUES(Calendar[Year]))"
        if include_year else f"SUMMARIZE(Products, Products[{hierby}])"
    )
    summarizegrand_expr = (
        f"CROSSJOIN(SUMMARIZE(Products, Products[Category]), VALUES(Calendar[Year]))"
        if include_year else f"SUMMARIZE(Products, Products[Category])"
    )
 
    dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                {summarize_expr},
                {column_exprs}
            ),
            TREATAS({years}, Calendar[Year]),
            {filters},
            TREATAS({{"{entity_name}"}}, Market[{area}]),
            FILTER('Scope', 'Scope'[Scope] = "{hierby}")
        )
    """
 
    totalcol_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                VALUES(Products[{hierby}]),
                {column_exprs}
            ),
            {filters},
            TREATAS({{"{entity_name}"}}, Market[{area}]),
            FILTER('Scope', 'Scope'[Scope] = "Category")
        )
    """
 
    grandtotal_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                {summarizegrand_expr},
                {column_exprs}
            ),
            TREATAS({years}, Calendar[Year]),
            {filters},
            TREATAS({{"{entity_name}"}}, Market[{area}]),
            FILTER('Scope', 'Scope'[Scope] = "Category")
        )
    """
 
    try:
        # Execute main query
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(dax_query)
            columns = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()
 
        # Execute total column query
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(totalcol_query)
            total_columns = [desc[0] for desc in cursor.description]
            total_data = cursor.fetchall()
 
        # Execute grand total query
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(grandtotal_query)
            grandtotal_columns = [desc[0] for desc in cursor.description]
            grandtotal_data = cursor.fetchall()
 
        # Main dataframe
        df = pd.DataFrame(data, columns=columns)
        df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)
        df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]
 
        # Total column dataframe
        dt = pd.DataFrame(total_data, columns=total_columns)
        dt.columns = dt.columns.str.replace(r'.*\[|\]', '', regex=True)
        dt = dt.loc[~(dt.select_dtypes(include='number') == 0).all(axis=1)]
 
        # Grand total dataframe
        grand_tot = pd.DataFrame(grandtotal_data, columns=grandtotal_columns)
        grand_tot.columns = grand_tot.columns.str.replace(r'.*\[|\]', '', regex=True)
        grand_tot = grand_tot.loc[~(grand_tot.select_dtypes(include='number') == 0).all(axis=1)]
       
        grand_tot[df.columns.difference(["Year", df.columns[0]])] = grand_tot[
            df.columns.difference(["Year", df.columns[0]])
        ].astype(float)
        grand_tot[df.columns[0]] = 'Grand Total'
 
        if 'Year' in df.columns and 'Year' not in grand_tot.columns:
            grand_tot['Year'] = df['Year'].max()
 
        grand_tot = grand_tot[df.columns]
        df = pd.concat([df, grand_tot], ignore_index=True)
 
        if dt.shape[1] > 0:
            dt.rename(columns=lambda col: f"Total {col}" if col != dt.columns[0] else col, inplace=True)
 
        if "Year" in df.columns:
            df["Year"] = pd.to_numeric(df["Year"], errors="coerce").astype("Int64")
 
 
        outputdic[key] = df
        outputtotal[key] = dt
 
        print(f"Query executed successfully for {entity_name}.")
 
    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_name} in {area}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_name} in {area}: {e}")
 
    return outputdic, outputtotal
 
 
def process_dax_queries(entity_hierarchy, hierarchy_levels, time_filter):
    with ThreadPoolExecutor(max_workers=8) as executor:
        dfs_results = {}
        dfs_totals = {}
        futures = []
        ordered_keys = []
        future_to_key = {}
        dfs_results = {}
        # futures = []
        for hierby, hier_values in hierarchy_levels:
            if hierby == "Category":
                continue
            for area, entity_list in entity_hierarchy:
                for entity in entity_list:
                    include_year = not time_filter
                    future = executor.submit(execute_dax_query, entity, area, hierby, time_filter, include_year=include_year)
                    key=f"{categories[0]} | {entity}"
                    ordered_keys.append(key)
                    futures.append(future)
                    future_to_key[future] = key    
 
       
       
            temp_results = {}
            temp_totals={}
            for future in as_completed(futures):
                result, total = future.result()
                temp_results.update(result)
                temp_totals.update(total)
               
            for key in ordered_keys:
                if key in temp_results:
                    dfs_results[key] = temp_results[key]
                    dfs_totals[key] = temp_totals[key]
 
 
 
            # Construct the output file name correctly
            filename = f"{hierby}_dfs.pkl"
            filenametotal =  f"{hierby}_total_dfs.pkl"
           
            output_file = f"{path}\\{filename}"
            with open(output_file, "wb") as f:
                pd.to_pickle(dfs_results, f)
               
            output_filetotal = f"{path}\\{filenametotal}"
            with open(output_filetotal, "wb") as f:
                pd.to_pickle(dfs_totals, f)
 
            print(f"All DataFrames saved to {output_file}.")
 
process_dax_queries(entity_hierarchy, hierarchy_levels,time_filter=False)  # sectors_dfs.pkl

Query executed successfully for Conv.
Query executed successfully for Bj's Corp.
Query executed successfully for Sam's Corp.
Query executed successfully for Drug.
Query executed successfully for Food.
Query executed successfully for NATIONAL.
Query executed successfully for Walmart.
All DataFrames saved to c:\Users\Ali Salem\Desktop\App_Update\Landscape\Landscape Datasets Test\Sector_dfs.pkl.
Query executed successfully for Bj's Corp.
Query executed successfully for Conv.
Query executed successfully for Food.
Query executed successfully for Sam's Corp.
Query executed successfully for Walmart.
Query executed successfully for Drug.
Query executed successfully for NATIONAL.
All DataFrames saved to c:\Users\Ali Salem\Desktop\App_Update\Landscape\Landscape Datasets Test\Segment_dfs.pkl.
Query executed successfully for Sam's Corp.
Query executed successfully for Conv.
Query executed successfully for Drug.
Query executed successfully for Walmart.
Query executed successfully for Bj's Corp.
Que

#### Client SCOPE = hierarchy

In [21]:
def execute_dax_query(entity_name, area, hierby, filter_p12m, brand=None, manuf=None):
    outputdic = {}
    
    time_filter = """
        FILTER(
            VALUES('Time Logic'[Time Period]), 
            'Time Logic'[Time Period] = "P12M"
        )
    """ if filter_p12m else ""

    manuf_filter = f'Products[{ManufOrTopC}]="{manuf}"' if manuf else ""
    brand_filter = f'Products[{BrandOrTopB}]="{brand}", Products[{ManufOrTopC}]="{client_manuf[0]}"' if brand else ""
    key = f"{categories[0]} | {brand} | {entity_name}" if brand else f"{categories[0]} | {manuf} | {entity_name}" if manuf else ""
    filters = ", ".join(filter(None, [manuf_filter, brand_filter, time_filter]))

    # Define columns dynamically
    columns = [
         "Value Share","Av Price/KG","Value Share DYA"
    ]
    
    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)


# Construct the DAX query dynamically
    dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(Products, Products[{hierby}]),
                {column_exprs}
            ),
            TREATAS({years}, Calendar[Year]),
            {filters},
            Products[Category] = "{categories[0]}",
            TREATAS({{"{entity_name}"}}, Market[{area}]),
            FILTER('Scope', 'Scope'[Scope] = "{hierby}")
        )   
    """

    try:
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(dax_query)
            columns = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()

            df = pd.DataFrame(data, columns=columns)
            df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)         
            df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]  # Remove zero rows
            if not df.empty:
                numeric_cols = df.select_dtypes(include='number').columns
                total_values = {}

                for col in numeric_cols:
                    if "price" in col.lower():  # Check if the column name contains "price" (case insensitive)
                        total_values[col] = df[col].mean()  # Take the average
                    else:
                        total_values[col] = df[col].sum()  # Take the sum

                grand_total_row = pd.DataFrame([["Grand Total"] + [total_values.get(col, " ") for col in df.columns[1:]]], columns=df.columns)
                df = pd.concat([df, grand_total_row], ignore_index=True)

            outputdic[key] = df
            
            print(f"Query executed successfully for {key}.")
    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_name} in {area}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_name} in {area}: {e}")

    return outputdic



def process_dax_queries(entity_hierarchy, hierarchy_levels, time_filter, client_brands=None, client_manuf=None):
    with ThreadPoolExecutor(max_workers=8) as executor:
        futures = []
        ordered_keys = []
        future_to_key = {}
        dfs_results = {}
        
        for hierby, hier_values in hierarchy_levels:
            if hierby == "Category":
                continue
            for area, entity_list in entity_hierarchy:
                for entity in entity_list:                    
                    for brand in client_brands:
                        future = executor.submit(execute_dax_query, entity, area, hierby, time_filter, brand=brand)
                        key = f"{categories[0]} | {brand} | {entity}"
                        ordered_keys.append(key)
                        futures.append(future)
                        future_to_key[future] = key  # ✅ add this line
                    for manuf in client_manuf:
                        future = executor.submit(execute_dax_query, entity, area, hierby, time_filter, manuf=manuf)
                        key=f"{categories[0]} | {manuf} | {entity}"
                        ordered_keys.append(key)
                        futures.append(future)
                        future_to_key[future] = key    

            temp_results = {}
            for future in as_completed(future_to_key):
                key = future_to_key[future]
                try:
                    result = future.result()
                    temp_results.update(result)
                except Exception as e:
                    print(f"Failed to retrieve result for {key}: {e}")

            # Preserve original key order
            for key in ordered_keys:
                if key in temp_results:
                    dfs_results[key] = temp_results[key]

            # Construct the output file name correctly
            if client_manuf:
                filename = f"{hierby}_client_dfs.pkl" if time_filter else f"{hierby}_dfs.pkl"

            output_file = f"{path}\\{filename}"

            with open(output_file, "wb") as f:
                pd.to_pickle(dfs_results, f)

            print(f"All DataFrames saved to {output_file}.")


process_dax_queries(entity_hierarchy,hierarchy_levels, time_filter=True,client_brands=client_brands,client_manuf=client_manuf)  # Brand-level



Query executed successfully for Manual Shave Men | Cremo | NATIONAL.
Query executed successfully for Manual Shave Men | Schick | NATIONAL.
Query executed successfully for Manual Shave Men | Pbg | Conv.
Query executed successfully for Manual Shave Men | Pbg | NATIONAL.
Query executed successfully for Manual Shave Men | Equate | NATIONAL.
Query executed successfully for Manual Shave Men | Equate | Conv.
Query executed successfully for Manual Shave Men | Schick | Conv.
Query executed successfully for Manual Shave Men | Cremo | Conv.
Query executed successfully for Manual Shave Men | Equate | Food.
Query executed successfully for Manual Shave Men | Schick | Food.
Query executed successfully for Manual Shave Men | Schick | Drug.
Query executed successfully for Manual Shave Men | Cremo | Food.
Query executed successfully for Manual Shave Men | Cremo | Drug.
Query executed successfully for Manual Shave Men | Pbg | Food.
Query executed successfully for Manual Shave Men | Equate | Drug.
Query e

## MANUFACTURER Dataframes& BRANDS Dataframes

In [22]:
def execute_dax_query(BrandorManuf,entity_name, area, hierby, filter_p12m, entity_type,include_year=False):
    outputdic = {}
    outputtotal={}
    time_filter = """
        FILTER(
            VALUES('Time Logic'[Time Period]), 
            'Time Logic'[Time Period] = "P12M"
        )
    """ if filter_p12m else ""
    key =  f"{entity_type} | {entity_name}"
    filters = ", ".join(filter(None, [f'Products[Category] = "{categories[0]}"', time_filter]))

    # Define columns dynamically
    columns = [
        "Volume Sales", "Value Sales", "Value Share", "Volume Share", "Share DYA","Av Price/KG","IYA Price/KG"
    ]
    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)

    dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                {f"CROSSJOIN(SUMMARIZE(Products, Products[{BrandorManuf}]), VALUES(Calendar[Year]))" if include_year else f"SUMMARIZE(Products, Products[{BrandorManuf}])"},
                {column_exprs}
            ),
            Products[{hierby}] = "{entity_type}",
            TREATAS({years}, Calendar[Year]),
            {filters},
            TREATAS({{"{entity_name}"}}, Market[{area}]),
            FILTER('Scope', 'Scope'[Scope] = "{hierby}")
        )   
    """
    totalcol_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                {f"Values(Products[{BrandorManuf}])"},
                {column_exprs}
            ),
            Products[{hierby}] = "{entity_type}",
            TREATAS({years}, Calendar[Year]),
            {filters},
            TREATAS({{"{entity_name}"}}, Market[{area}]),
            FILTER('Scope', 'Scope'[Scope] = "Category")
        )   
    """
    grand_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                {f"CROSSJOIN(SUMMARIZE(Products, Products[Category]), VALUES(Calendar[Year]))" },
                {column_exprs}
            ),
            Products[{hierby}] = "{entity_type}",
            TREATAS({years}, Calendar[Year]),
            {filters},
            TREATAS({{"{entity_name}"}}, Market[{area}]),
            FILTER('Scope', 'Scope'[Scope] = "Category")
        )   
    """
    try:
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(dax_query)
            columns = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(totalcol_query)
            totalcol_columns = [desc[0] for desc in cursor.description]
            totalcol_data = cursor.fetchall()
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(grand_query)
            grandtotal_columns = [desc[0] for desc in cursor.description]
            grandtotal_data = cursor.fetchall()
            
            df = pd.DataFrame(data, columns=columns)
            df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)         
            df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]  # Remove zero rows
            
               
            dt = pd.DataFrame(totalcol_data, columns=totalcol_columns)
            dt.columns = dt.columns.str.replace(r'.*\[|\]', '', regex=True)
            dt = dt.loc[~(dt.select_dtypes(include='number') == 0).all(axis=1)]  # Remove zero rows
            
            grand_tot = pd.DataFrame(grandtotal_data, columns=grandtotal_columns)
            grand_tot.columns = grand_tot.columns.str.replace(r'.*\[|\]', '', regex=True)
            grand_tot = grand_tot.loc[~(grand_tot.select_dtypes(include='number') == 0).all(axis=1)]  # Remove zero rows
        

            grand_tot[df.columns.difference(["Year", df.columns[0]])] = grand_tot[df.columns.difference(["Year", df.columns[0]])].astype(float)

            # Optionally add a label for clarity
            grand_tot[df.columns[0]] = 'Grand Total'
            if 'Year' in df.columns and 'Year' not in grand_tot.columns:
                grand_tot['Year'] = df['Year'].max()  # or some other default like np.nan or a fixed value

            # Reorder columns if necessary
            grand_tot = grand_tot[df.columns]

            # Concatenate the two
            df = pd.concat([df, grand_tot], ignore_index=True)

            if dt.shape[1] > 0:
                dt.rename(columns=lambda col: f"Total {col}" if col != dt.columns[0] else col, inplace=True)

            if "Year" in df.columns:
                df["Year"] = df["Year"].astype(int)  # Ensure Year is integer

            outputdic[key] = df # Always store result
            outputtotal[key]=dt

            
            print(f"Query executed successfully for {entity_name}.")
    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_name} in {area}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_name} in {area}: {e}")

    return outputdic,outputtotal

def process_dax_queries(BrandorManuf,entity_hierarchy, hierarchy_levels, time_filter):
    
    with ThreadPoolExecutor(max_workers=8) as executor:
        futures = []
        ordered_keys=[]
        dfs_results = {}
        dfs_totals={}
        for hierby, hier_values in hierarchy_levels:
            for value in hier_values:
                for area, entity_list in entity_hierarchy:
                    for entity in entity_list:
                        include_year = not time_filter 
                        key = f"{value} | {entity}"
                        ordered_keys.append(key)
                        future = executor.submit(execute_dax_query,BrandorManuf,entity, area, hierby, time_filter,value, include_year=include_year)
                        futures.append(future)

            temp_results = {}
            temp_totals={}
            for future in as_completed(futures):
                result, total = future.result()
                temp_results.update(result)
                temp_totals.update(total)
                
            for key in ordered_keys:
                if key in temp_results:
                    dfs_results[key] = temp_results[key]
                    dfs_totals[key] = temp_totals[key]
                

        # Construct the output file name correctly
        if BrandorManuf==f'{ManufOrTopC}':
            filename = "manuf_dfs.pkl"
            filenametotal = "manuf_total_dfs.pkl"
            
        else:
            filename = "brand_12_dfs.pkl" if time_filter else "brand_dfs.pkl"
            filenametotal = "brand_12_total_dfs.pkl" if time_filter else "brand_total_dfs.pkl"

               
        output_file = f"{path}\\{filename}"
        with open(output_file, "wb") as f:
            pd.to_pickle(dfs_results, f)
            
        output_filetotal = f"{path}\\{filenametotal}"
        with open(output_filetotal, "wb") as f:
            pd.to_pickle(dfs_totals, f)    

            print(f"All DataFrames saved to {output_file}.")
process_dax_queries(f'{ManufOrTopC}',entity_hierarchy, hierarchy_levels,time_filter=False)  
process_dax_queries(f'{BrandOrTopB}',entity_hierarchy,hierarchy_levels, time_filter=False)   # sectors_12_dfs.pkl
process_dax_queries(f'{BrandOrTopB}',entity_hierarchy,hierarchy_levels, time_filter=True)  



Query executed successfully for Conv.
Query executed successfully for Drug.
Query executed successfully for Bj's Corp.
Query executed successfully for Sam's Corp.
Query executed successfully for Food.
Query executed successfully for NATIONAL.
Query executed successfully for Walmart.
Query executed successfully for Bj's Corp.
Query executed successfully for Sam's Corp.
Query executed successfully for Drug.
Query executed successfully for Conv.
Query executed successfully for Walmart.
Query executed successfully for NATIONAL.
Query executed successfully for NATIONAL.
Query executed successfully for Food.
Query executed successfully for Conv.
Query executed successfully for Sam's Corp.
Query executed successfully for Drug.
Query executed successfully for Walmart.
Query executed successfully for Bj's Corp.
Query executed successfully for Food.
Query executed successfully for Sam's Corp.
Query executed successfully for Drug.
Query executed successfully for Conv.
Query executed successfully 

MANUFACTURER Dataframes"P12M", "LY" 

In [23]:
def execute_dax_query(BrandorManuf, entity_name, area, hierby, entity_type=None):
    outputdic = {}
    outputtotal={}
    time_periods = '{"P12M", "LY"}'  # Correct format for DAX

    key = f"{entity_type} | {entity_name}"
    columns = ["Volume Sales", "Value Sales", "Value Share", "Volume Share", "Share DYA", "Av Price/KG"]
    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)

    dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                CROSSJOIN(
                    SUMMARIZE(Products, Products[{BrandorManuf}]), 
                    VALUES('Time Logic'[Time Period])
                ),
                {column_exprs}
            ),
            Products[{hierby}] = "{entity_type}",
            TREATAS({time_periods}, 'Time Logic'[Time Period]),
            Products[Category] = "{categories[0]}",
            TREATAS({{"{entity_name}"}}, Market[{area}]),
            FILTER('Scope', 'Scope'[Scope] = "{hierby}")
        )   
    """
    grand_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                {f"Values(Products[{BrandorManuf}])"},
                {column_exprs}
            ),
            Products[{hierby}] = "{entity_type}",
            TREATAS({time_periods}, 'Time Logic'[Time Period]),
            Products[Category] = "{categories[0]}",
            TREATAS({{"{entity_name}"}}, Market[{area}]),
            FILTER('Scope', 'Scope'[Scope] = "{hierby}")
        )   
    """
    try:
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(dax_query)
            columns = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()
            
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(grand_query)
            grand_columns = [desc[0] for desc in cursor.description]
            grand_data = cursor.fetchall()   
            df = pd.DataFrame(data, columns=columns)
            df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)         
            df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]  # Remove zero rows
            
            dt = pd.DataFrame(grand_data, columns=grand_columns)
            dt.columns = dt.columns.str.replace(r'.*\[|\]', '', regex=True)         
            dt = pd.DataFrame(grand_data, columns=grand_columns)
            dt.columns = dt.columns.str.replace(r'.*\[|\]', '', regex=True)
            dt = dt.loc[~(dt.select_dtypes(include='number') == 0).all(axis=1)]  # Remove zero rows

            if dt.shape[1] > 0:
                dt.rename(columns=lambda col: f"Total {col}" if col != dt.columns[0] else col, inplace=True)

            if "Year" in df.columns:
                df["Year"] = df["Year"].astype(int)  # Ensure Year is integer

            outputdic[key] = df  # Always store result
            outputtotal[key]=dt
            
            print(f"Query executed successfully for {entity_name}.")
    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_name} in {area}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_name} in {area}: {e}")

    return outputdic,outputtotal


def process_dax_queries(BrandorManuf, entity_hierarchy, hierarchy_levels):
    dfs_results = {}
    dfs_totals={}
    with ThreadPoolExecutor(max_workers=8) as executor:
        futures = []
        ordered_keys=[]
        for hierby, hier_values in hierarchy_levels:
            for value in hier_values:   
                for area, entity_list in entity_hierarchy:
                    for entity in entity_list:
                        key = f"{value} | {entity}"
                        ordered_keys.append(key)
                        future = executor.submit(execute_dax_query, BrandorManuf, entity, area, hierby, value)
                        futures.append(future)
            temp_results = {}
            temp_totals={}
            for future in as_completed(futures):
                result, total = future.result()
                temp_results.update(result)
                temp_totals.update(total)
                
            for key in ordered_keys:
                if key in temp_results:
                    dfs_results[key] = temp_results[key]
                    dfs_totals[key] = temp_totals[key] 

        # Construct the output file name correctly
        filename = "manuf_12_dfs.pkl"
        filenametotal="manuf_12_total_dfs.pkl"
        output_file = f"{path}\\{filename}"
        with open(output_file, "wb") as f:
            pd.to_pickle(dfs_results, f)
            
        output_filetotal = f"{path}\\{filenametotal}"
        with open(output_filetotal, "wb") as f:
            pd.to_pickle(dfs_totals, f) 

        print(f"All DataFrames saved to {output_file}.")


process_dax_queries(ManufOrTopC, entity_hierarchy, hierarchy_levels)


Query executed successfully for Bj's Corp.
Query executed successfully for Food.
Query executed successfully for Drug.
Query executed successfully for NATIONAL.
Query executed successfully for Conv.
Query executed successfully for Sam's Corp.
Query executed successfully for Walmart.
Query executed successfully for Bj's Corp.
Query executed successfully for Sam's Corp.
Query executed successfully for Conv.
Query executed successfully for Drug.
Query executed successfully for NATIONAL.
Query executed successfully for Food.
Query executed successfully for Walmart.
Query executed successfully for NATIONAL.
Query executed successfully for Conv.
Query executed successfully for Food.
Query executed successfully for Sam's Corp.
Query executed successfully for Drug.
Query executed successfully for Walmart.
Query executed successfully for Bj's Corp.
Query executed successfully for Sam's Corp.
Query executed successfully for Bj's Corp.
Query executed successfully for NATIONAL.
Query executed succ

#### Manuf&Brands Evolution 

In [24]:
def execute_dax_query(BrandorManuf,entity_name, area, hierby, entity_type):
    outputdic = {}

    key =  f"{entity_type} | {entity_name}"

    # Define columns dynamically
    columns = [
        "Value Sales", "Price Evolution", "Share Evolution"
    ]
    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)

    dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                CROSSJOIN(
                    VALUES(Calendar[Year]),
                    SUMMARIZE(Products, Products[{BrandorManuf}])
                ),
                {column_exprs}
            ),
            FILTER('Scope', 'Scope'[Scope] = "{hierby}"),
            Products[{hierby}] = "{entity_type}",
            FILTER(Products, Products[Category] = "{categories[0]}"),
            TREATAS({{"{entity_name}"}}, Market[{area}])
        )
  
    """
    
    try:
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(dax_query)
            columns = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()

            df = pd.DataFrame(data, columns=columns)
            df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)      
               
            df = df.loc[~(df.iloc[:, 1:].select_dtypes(include='number') == 0).all(axis=1)]  # Remove zero rows
            
            if "Year" in df.columns:
                df = df[df["Year"].notna()]
                df["Year"] = df["Year"].astype(int)  # Ensure Year is integer
            
            outputdic[key] = df
            
            print(f"Query executed successfully for {entity_name}.")
    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_name} in {area}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_name} in {area}: {e}")
    return outputdic

def process_dax_queries(BrandorManuf,entity_hierarchy, hierarchy_levels):
    dfs_results = {}
    with ThreadPoolExecutor(max_workers=8) as executor:
        futures = []
        ordered_keys=[]
        future_to_key={}
        for hierby, hier_values in hierarchy_levels:
            for value in hier_values:
                for area, entity_list in entity_hierarchy:
                    for entity in entity_list:
                        future = executor.submit(execute_dax_query,BrandorManuf,entity, area, hierby,value)
                        key = f"{value} | {entity}"
                        ordered_keys.append(key)
                        futures.append(future)
                        future_to_key[future] = key    

        temp_results = {}
        for future in as_completed(future_to_key):
            key = future_to_key[future]
            try:
                result = future.result()
                temp_results.update(result)
            except Exception as e:
                print(f"Failed to retrieve result for {key}: {e}")

        # Preserve original key order
        for key in ordered_keys:
            if key in temp_results:
                dfs_results[key] = temp_results[key]

                

        # Construct the output file name correctly
        if BrandorManuf==f'{ManufOrTopC}':
            filename = "manuf_evolution.pkl"
        else:
            filename = "brands_evolution.pkl"
               
        output_file = f"{path}\\{filename}"
        with open(output_file, "wb") as f:
            pd.to_pickle(dfs_results, f)

            print(f"All DataFrames saved to {output_file}.")
process_dax_queries(f'{ManufOrTopC}',entity_hierarchy, hierarchy_levels)
process_dax_queries(f'{BrandOrTopB}',entity_hierarchy, hierarchy_levels)  


Query executed successfully for Sam's Corp.
Query executed successfully for Conv.
Query executed successfully for Walmart.
Query executed successfully for Drug.
Query executed successfully for NATIONAL.
Query executed successfully for Bj's Corp.
Query executed successfully for Food.
Query executed successfully for NATIONAL.
Query executed successfully for Sam's Corp.
Query executed successfully for Conv.
Query executed successfully for Bj's Corp.
Query executed successfully for Food.
Query executed successfully for Walmart.
Query executed successfully for Drug.
Query executed successfully for NATIONAL.
Query executed successfully for Conv.
Query executed successfully for Drug.
Query executed successfully for Food.
Query executed successfully for Sam's Corp.
Query executed successfully for Bj's Corp.
Query executed successfully for Walmart.
Query executed successfully for NATIONAL.
Query executed successfully for Conv.
Query executed successfully for Food.
Query executed successfully fo

In [25]:
def execute_dax_query(BrandorManuf,entity_name, area, hierby, entity_type):
    outputdic = {}

    key =  f"{entity_type} | {entity_name}"

    # Define columns dynamically
    columns = [
        "Value Sales", "Price Evolution", "Share Evolution"
    ]
    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)

    dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                VALUES(Calendar[Year]),
                {column_exprs}
            ),
            FILTER('Scope', 'Scope'[Scope] = "{hierby}"),
            Products[{hierby}] = "{entity_type}",
            FILTER(Products, Products[Category] = "{categories[0]}"),
            TREATAS({{"{entity_name}"}}, Market[{area}])
        )
  
    """
    
    try:
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(dax_query)
            columns = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()

            df = pd.DataFrame(data, columns=columns)
            df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)      
               
            df = df.loc[~(df.iloc[:, 1:].select_dtypes(include='number') == 0).all(axis=1)]  # Remove zero rows
            
            if "Year" in df.columns:
                df = df[df["Year"].notna()]
                df["Year"] = df["Year"].astype(int)  # Ensure Year is integer
            
            outputdic[key] = df
            
            print(f"Query executed successfully for {entity_name}.")
    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_name} in {area}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_name} in {area}: {e}")
    return outputdic

def process_dax_queries(BrandorManuf,entity_hierarchy, hierarchy_levels):
    dfs_results = {}
    with ThreadPoolExecutor(max_workers=8) as executor:
        futures = []
        ordered_keys=[]
        future_to_key={}
        for hierby, hier_values in hierarchy_levels:
            for value in hier_values:
                for area, entity_list in entity_hierarchy:
                    for entity in entity_list:
                        future = executor.submit(execute_dax_query,BrandorManuf,entity, area, hierby,value)
                        key = f"{value} | {entity}"
                        ordered_keys.append(key)
                        futures.append(future)
                        future_to_key[future] = key  
                        
        temp_results = {}
        for future in as_completed(future_to_key):
            key = future_to_key[future]
            try:
                result = future.result()
                temp_results.update(result)
            except Exception as e:
                print(f"Failed to retrieve result for {key}: {e}")

        # Preserve original key order
        for key in ordered_keys:
            if key in temp_results:
                dfs_results[key] = temp_results[key]
                

        # Construct the output file name correctly
        if BrandorManuf==f'{ManufOrTopC}':
            filename = "manuf_evolution_total.pkl"
        else:
            filename = "brands_evolution_total.pkl"
               
        output_file = f"{path}\\{filename}"
        with open(output_file, "wb") as f:
            pd.to_pickle(dfs_results, f)

            print(f"All DataFrames saved to {output_file}.")
process_dax_queries(f'{ManufOrTopC}',entity_hierarchy, hierarchy_levels)
process_dax_queries(f'{BrandOrTopB}',entity_hierarchy, hierarchy_levels)  


Query executed successfully for Sam's Corp.
Query executed successfully for Walmart.
Query executed successfully for Drug.
Query executed successfully for Conv.
Query executed successfully for Bj's Corp.
Query executed successfully for NATIONAL.
Query executed successfully for Food.
Query executed successfully for NATIONAL.
Query executed successfully for Conv.
Query executed successfully for Drug.
Query executed successfully for Sam's Corp.
Query executed successfully for Food.
Query executed successfully for Bj's Corp.
Query executed successfully for Walmart.
Query executed successfully for Conv.
Query executed successfully for NATIONAL.
Query executed successfully for Conv.
Query executed successfully for Food.
Query executed successfully for Sam's Corp.
Query executed successfully for NATIONAL.
Query executed successfully for Bj's Corp.
Query executed successfully for Drug.
Query executed successfully for Food.
Query executed successfully for Walmart.
Query executed successfully fo

## Calendar Dataframes

In [26]:
def execute_dax_query(entity_name, area, hierby, entity_type=None, brand=None):
    outputdic = {}
    key = f"{brand} | {entity_name}" if brand else f"{entity_type} | {entity_name}"
    columns = ["Value Sales", "Av Price/KG"]

    # Handle brand filter correctly
    brand_filter = f'Products[{BrandOrTopB}]="{brand}"' if brand else None

    if hierby=="Category":
        filters = [
            f'TREATAS({{"{entity_name}"}}, Market[{area}])',
            "FILTER('Scope', 'Scope'[Scope] = \"{hierby}\")"
        ]
    else:    
        filters = [
        f'Products[{hierby}] = "{entity_type}"',
        f'TREATAS({{"{entity_name}"}}, Market[{area}])',
        "FILTER('Scope', 'Scope'[Scope] = \"{hierby}\")"
    ]
    if brand_filter:
        filters.append(brand_filter)

    # Ensure filters are correctly joined
    filter_clause = ",\n            ".join(filters)

    # Construct column expressions
    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)

    # Construct DAX query
    dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(
                    Calendar, 
                    Calendar[MonthYear]
                ),
                {column_exprs}
            ),
            {filter_clause}
        )
    """
    try:
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(dax_query)
            columns = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()

            df = pd.DataFrame(data, columns=columns)
            df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)  # Clean column names
            df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]  # Remove zero rows
            if not df.empty:
                    numeric_cols = df.select_dtypes(include='number').columns
                    total_values = {}

                    for col in numeric_cols:
                        if "price" in col.lower():  # Check if the column name contains "price" (case insensitive)
                            total_values[col] = df[col].mean()  # Take the average
                        else:
                            total_values[col] = df[col].sum()  # Take the sum
                    grand_total_row = pd.DataFrame([["Grand Total"] + [total_values.get(col, " ") for col in df.columns[1:]]], columns=df.columns)
                    df = pd.concat([df, grand_total_row], ignore_index=True)
            outputdic[key] = df
            print(f"Query executed successfully for {entity_name}.")
    
    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_name} in {area}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_name} in {area}: {e}")

    return outputdic


def process_dax_queries(entity_hierarchy, hierarchy_levels, client_brands, path):
    dfs_results = {}

    with ThreadPoolExecutor(max_workers=8) as executor:
        futures = []
        ordered_keys=[]
        future_to_key={}
        for hierby, hier_values in hierarchy_levels:  # Ensure hierarchy_levels is a dict
            for value in hier_values:
                for area, entity_list in entity_hierarchy:  # Ensure it's dict.items()
                    for entity in entity_list:
                        future = executor.submit(execute_dax_query, entity, area, hierby, value)
                        key = f"{value} | {entity}"
                        ordered_keys.append(key)
                        futures.append(future)
                        future_to_key[future] = key  
                        for brand in client_brands:
                            if hierby =="Category":
                                future = executor.submit(execute_dax_query, entity, area, hierby, value, brand=brand)
                                key = f"{brand} | {entity}" 
                                ordered_keys.append(key)
                                futures.append(future)
                                future_to_key[future] = key  
        temp_results = {}
        for future in as_completed(future_to_key):
            key = future_to_key[future]
            try:
                result = future.result()
                temp_results.update(result)
            except Exception as e:
                print(f"Failed to retrieve result for {key}: {e}")

        # Preserve original key order
        for key in ordered_keys:
            if key in temp_results:
                dfs_results[key] = temp_results[key]

    # Save results to a pickle file
    output_file = f"{path}\\calendar_dfs.pkl"
    with open(output_file, "wb") as f:
        pd.to_pickle(dfs_results, f)

    print(f"All DataFrames saved to {output_file}.")


# Run function (ensure variables are defined before calling)
process_dax_queries(entity_hierarchy, hierarchy_levels, client_brands, path)


Query executed successfully for Conv.
Query executed successfully for Conv.
Query executed successfully for Conv.
Query executed successfully for Conv.
Query executed successfully for NATIONAL.
Query executed successfully for NATIONAL.
Query executed successfully for NATIONAL.
Query executed successfully for NATIONAL.
Query executed successfully for Food.
Query executed successfully for Food.
Query executed successfully for Food.
Query executed successfully for Drug.
Query executed successfully for Drug.
Query executed successfully for Drug.
Query executed successfully for Drug.
Query executed successfully for Food.
Query executed successfully for Sam's Corp.
Query executed successfully for Sam's Corp.
Query executed successfully for Sam's Corp.
Query executed successfully for Sam's Corp.
Query executed successfully for Walmart.
Query executed successfully for Walmart.
Query executed successfully for Walmart.
Query executed successfully for Walmart.
Query executed successfully for Bj's

### Category Overview Dataframes

In [27]:
def execute_dax_query(entity_name, area, hierby, entity_type=None, manuf=None):
    outputdic = {}
    key = f"{entity_type} | {entity_name}"
    columns = ["Volume Sales","Value Sales","Volume Sales IYA","Value Sales IYA","IYA Price/KG"]

    # Handle brand filter correctly
    manuf_filter = f'FILTER(Products,Products[{ManufOrTopC}]="{manuf}"),' if manuf else ""
    filters = ", ".join(filter(None, [manuf_filter]))


    # Construct DAX query
    dax_query = f"""
        EVALUATE
        SUMMARIZECOLUMNS(
            FILTER(Products,Products[{hierby}] = "{entity_type}"),
            FILTER(Products, Products[Category] = "{categories[0]}"),
            {manuf_filter}
            TREATAS({{"{entity_name}"}}, Market[{area}]),
            FILTER('Time Logic', 'Time Logic'[Time Period] = "P12M"),
            FILTER('Scope', 'Scope'[Scope] = "Category"),
            "Volume Sales", COALESCE([Volume Sales], 0),
            "Value Sales", COALESCE([Value Sales], 0),
            "Volume Sales IYA", COALESCE([Volume Sales IYA], 0),
            "Value Sales IYA", COALESCE([Value Sales IYA], 0),
            "IYA Price/KG", COALESCE([IYA Price/KG], 0)
        )

    """
    try:
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(dax_query)
            columns = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()

            df = pd.DataFrame(data, columns=columns)
            df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)  # Clean column names
            df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]  # Remove zero rows
        
            outputdic[key] = df
            print(f"Query executed successfully for {entity_name}.")
    
    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_name} in {area}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_name} in {area}: {e}")

    return outputdic


def process_dax_queries(entity_hierarchy, hierarchy_levels, path,client_manuf=None):
    dfs_results = {}

    with ThreadPoolExecutor(max_workers=8) as executor:
        futures = []
        ordered_keys=[]
        future_to_key={}
        for hierby, hier_values in hierarchy_levels:  # Ensure hierarchy_levels is a dict
            if hierby=="Category":
                for value in hier_values:
                    for area, entity_list in entity_hierarchy:  # Ensure it's dict.items()
                        for entity in entity_list:
                            if client_manuf:
                                for manuf in client_manuf:
                                    future = executor.submit(execute_dax_query, entity, area, hierby, value, manuf=manuf)
                                    key = f"{value} | {entity}" 
                                    ordered_keys.append(key)
                                    futures.append(future)
                                    future_to_key[future] = key  
                            else:
                                future = executor.submit(execute_dax_query, entity, area, hierby, value)
                                key = f"{value} | {entity}" 
                                ordered_keys.append(key)
                                futures.append(future)
                                future_to_key[future] = key  
        temp_results = {}
        for future in as_completed(future_to_key):
            key = future_to_key[future]
            try:
                result = future.result()
                temp_results.update(result)
            except Exception as e:
                print(f"Failed to retrieve result for {key}: {e}")

        # Preserve original key order
        for key in ordered_keys:
            if key in temp_results:
                dfs_results[key] = temp_results[key]

    # Save results to a pickle file
    output_file =f"{path}\\categories_overview_manuf_dfs.pkl" if client_manuf else f"{path}\\categories_overview_dfs.pkl"
    with open(output_file, "wb") as f:
        pd.to_pickle(dfs_results, f)

    print(f"All DataFrames saved to {output_file}.")

process_dax_queries(entity_hierarchy, hierarchy_levels, path)
process_dax_queries(entity_hierarchy, hierarchy_levels, path,client_manuf=client_manuf)


Query executed successfully for Conv.
Query executed successfully for Sam's Corp.
Query executed successfully for Food.
Query executed successfully for Bj's Corp.
Query executed successfully for Drug.
Query executed successfully for NATIONAL.
Query executed successfully for Walmart.
All DataFrames saved to c:\Users\Ali Salem\Desktop\App_Update\Landscape\Landscape Datasets Test\categories_overview_dfs.pkl.
Query executed successfully for Sam's Corp.
Query executed successfully for Conv.
Query executed successfully for Bj's Corp.
Query executed successfully for Food.
Query executed successfully for Drug.
Query executed successfully for Walmart.
Query executed successfully for NATIONAL.
All DataFrames saved to c:\Users\Ali Salem\Desktop\App_Update\Landscape\Landscape Datasets Test\categories_overview_manuf_dfs.pkl.


### Category Values Dataframes

In [28]:
def execute_dax_query(entity_name, area, hierby, entity_type=None):
    outputdic = {}
    key = f"{entity_type} | {entity_name}"
    columns = ["Value Sales"]

    # Collect filters
    filters = [
        f'Products[Category] = "{categories[0]}"',
        f'TREATAS({{"{entity_name}"}}, Market[{area}])'
    ]
    # Ensure filters are correctly joined
    filter_clause = ",\n            ".join(filters)

    # Construct column expressions
    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)

    # Construct DAX query
    dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(
                CROSSJOIN(Products, Calendar),
                            Calendar[MonthYear],                
                            Products[Sector]
                ),
                {column_exprs}
            ),
            {filter_clause}
        )
    """
    try:
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(dax_query)
            columns = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()

            df = pd.DataFrame(data, columns=columns)
            df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)  # Clean column names
            df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]  # Remove zero rows
            if not df.empty:
                    numeric_cols = df.select_dtypes(include='number').columns
                    total_values = {}

                    for col in numeric_cols:
                        if "price" in col.lower():  # Check if the column name contains "price" (case insensitive)
                            total_values[col] = df[col].mean()  # Take the average
                        else:
                            total_values[col] = df[col].sum()  # Take the sum
                    grand_total_row = pd.DataFrame([["Grand Total"] + [total_values.get(col, " ") for col in df.columns[1:]]], columns=df.columns)
                    df = pd.concat([df, grand_total_row], ignore_index=True)
            outputdic[key] = df
            print(f"Query executed successfully for {entity_name}.")
    
    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_name} in {area}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_name} in {area}: {e}")

    return outputdic


def process_dax_queries(entity_hierarchy, hierarchy_levels, path):
    dfs_results = {}

    with ThreadPoolExecutor(max_workers=8) as executor:
        futures = []
        ordered_keys=[]
        future_to_key={}
        for hierby, hier_values in hierarchy_levels:  # Ensure hierarchy_levels is a dict
            for value in hier_values:
                if hierby == "Category":
                    for area, entity_list in entity_hierarchy:  # Ensure it's dict.items()
                        for entity in entity_list:
                        # Submit task for each hierarchy level
                            future = executor.submit(execute_dax_query, entity, area, hierby, value)
                            key = f"{value} | {entity}" 
                            ordered_keys.append(key)
                            futures.append(future)
                            future_to_key[future] = key  

        temp_results = {}
        for future in as_completed(future_to_key):
            key = future_to_key[future]
            try:
                result = future.result()
                temp_results.update(result)
            except Exception as e:
                print(f"Failed to retrieve result for {key}: {e}")

        # Preserve original key order
        for key in ordered_keys:
            if key in temp_results:
                dfs_results[key] = temp_results[key]

    # Save results to a pickle file
    output_file = f"{path}\\categories_values_dfs.pkl"
    with open(output_file, "wb") as f:
        pd.to_pickle(dfs_results, f)

    print(f"All DataFrames saved to {output_file}.")


# Run function (ensure variables are defined before calling)
process_dax_queries(entity_hierarchy, hierarchy_levels, path)


Query executed successfully for Sam's Corp.
Query executed successfully for Conv.
Query executed successfully for Drug.
Query executed successfully for NATIONAL.
Query executed successfully for Bj's Corp.
Query executed successfully for Walmart.
Query executed successfully for Food.
All DataFrames saved to c:\Users\Ali Salem\Desktop\App_Update\Landscape\Landscape Datasets Test\categories_values_dfs.pkl.


## Client Sec,Seg,.. SCOPE = CATEGORY

In [29]:
def execute_dax_query(entity_name, area, hierby, filter_p12m, brand=None, manuf=None):
    outputdic = {}
    
    time_filter = """
        FILTER(
            VALUES('Time Logic'[Time Period]), 
            'Time Logic'[Time Period] = "P12M"
        )
    """ if filter_p12m else ""

    manuf_filter = f'Products[{ManufOrTopC}]="{manuf}"' if manuf else ""
    brand_filter = f'Products[{BrandOrTopB}]="{brand}", Products[{ManufOrTopC}]="{client_manuf[0]}"' if brand else ""
    key = f"{categories[0]} | {brand} | {entity_name}" if brand else f"{categories[0]} | {manuf} | {entity_name}" if manuf else ""
    filters = ", ".join(filter(None, [manuf_filter, brand_filter, time_filter]))

    # Define columns dynamically
    columns = [
        "Value Sales", "Value Share","Value Sales IYA","Av Price/KG","WoB %","Volume Sales IYA","IYA Price/KG"
    ]

    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)


# Construct the DAX query dynamically
    dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(Products, Products[{hierby}]),
                {column_exprs}
            ),
            TREATAS({years}, Calendar[Year]),
            {filters},
            Products[Category] = "{categories[0]}",
            TREATAS({{"{entity_name}"}}, Market[{area}]),
            FILTER('Scope', 'Scope'[Scope] = "Category")
        )   
    """
    try:
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(dax_query)
            columns = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()

            df = pd.DataFrame(data, columns=columns)
            df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)         
            df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]  # Remove zero rows
            if not df.empty:
                numeric_cols = df.select_dtypes(include='number').columns
                total_values = {}

                for col in numeric_cols:
                    if "price" in col.lower():  # Check if the column name contains "price" (case insensitive)
                        total_values[col] = df[col].mean()  # Take the average
                    else:
                        total_values[col] = df[col].sum()  # Take the sum

                grand_total_row = pd.DataFrame([["Grand Total"] + [total_values.get(col, " ") for col in df.columns[1:]]], columns=df.columns)
                df = pd.concat([df, grand_total_row], ignore_index=True)

            outputdic[key] = df
            
            print(f"Query executed successfully for {entity_name}.")
    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_name} in {area}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_name} in {area}: {e}")

    return outputdic



def process_dax_queries(entity_hierarchy, hierarchy_levels, time_filter, client_brands=None, client_manuf=None):
    dfs_results = {}
    with ThreadPoolExecutor(max_workers=8) as executor:
        futures = []
        ordered_keys=[]
        future_to_key={}
        for hierby, hier_values in hierarchy_levels:
            if hierby == "Category":
                continue
            for area, entity_list in entity_hierarchy:
                for entity in entity_list:                    
                    if client_brands:
                        for brand in client_brands:
                            future = executor.submit(execute_dax_query, entity, area, hierby, time_filter, brand=brand)
                            key = f"{categories[0]} | {brand} | {entity}" 
                            ordered_keys.append(key)
                            futures.append(future)
                            future_to_key[future] = key  
                        for manuf in client_manuf:
                            future = executor.submit(execute_dax_query, entity, area, hierby, time_filter, manuf=manuf)
                            key = f"{categories[0]} | {manuf} | {entity}" 
                            ordered_keys.append(key)
                            futures.append(future)
                            future_to_key[future] = key  

            temp_results = {}
            for future in as_completed(future_to_key):
                key = future_to_key[future]
                try:
                    result = future.result()
                    temp_results.update(result)
                except Exception as e:
                    print(f"Failed to retrieve result for {key}: {e}")

            # Preserve original key order
            for key in ordered_keys:
                if key in temp_results:
                    dfs_results[key] = temp_results[key]
            # Construct the output file name correctly
            if client_manuf:
                filename = f"{hierby}_client_dfs_category.pkl" if time_filter else f"{hierby}_dfs.pkl"

            output_file = f"{path}\\{filename}"

            with open(output_file, "wb") as f:
                pd.to_pickle(dfs_results, f)

            print(f"All DataFrames saved to {output_file}.")


process_dax_queries(entity_hierarchy,hierarchy_levels, time_filter=True,client_brands=client_brands,client_manuf=client_manuf)  # Brand-level

Query executed successfully for Conv.
Query executed successfully for NATIONAL.
Query executed successfully for NATIONAL.
Query executed successfully for Conv.
Query executed successfully for Conv.
Query executed successfully for NATIONAL.
Query executed successfully for Conv.
Query executed successfully for NATIONAL.
Query executed successfully for Food.
Query executed successfully for Food.
Query executed successfully for Food.
Query executed successfully for Food.
Query executed successfully for Drug.
Query executed successfully for Drug.
Query executed successfully for Drug.
Query executed successfully for Drug.
Query executed successfully for Sam's Corp.
Query executed successfully for Sam's Corp.
Query executed successfully for Sam's Corp.
Query executed successfully for Sam's Corp.
Query executed successfully for Walmart.
Query executed successfully for Walmart.
Query executed successfully for Walmart.
Query executed successfully for Walmart.
Query executed successfully for Bj's

## Retailers, Channels, Custom Dataframes

In [30]:
def execute_dax_query(hierby, filter_p12m, area ,entity_type=None, row_field=None):
    outputdic = {}

    time_filter = """
        FILTER(
            VALUES('Time Logic'[Time Period]), 
            'Time Logic'[Time Period] = "P12M"
        )
    """ if filter_p12m else ""

    key = f"{entity_type}"
    filters = ", ".join(filter(None, [time_filter]))

    columns = [
        "Trade WoB %", "Trade WoB % DYA", "Channel Growth Contribution",
        "Value Sales", "Value Sales IYA", "Av Price/KG", "Channel Relative Price",
        "Value Share", "IYA Price/KG", "Value Share DYA"
    ]
    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)

    # Construct DAX query dynamically with a single row_field
    dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(Market, {row_field}),
                {column_exprs}
            ),
            TREATAS({years}, Calendar[Year]),
            {filters},
            FILTER(Products, Products[{hierby}] = "{entity_type}"),
            FILTER(Market, Market[Area] = "{area}"),
            FILTER('Scope', 'Scope'[Scope] = "{hierby}")
        )
    """

    try:
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(dax_query)
            columns = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()

            df = pd.DataFrame(data, columns=columns)
            df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)         
            df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]  # Remove zero rows
            df.columns = df.columns.str.replace('Trade ', '').str.replace('Channel ','').str.strip()
            df = df.sort_values('WoB %', ascending=False)              
            if not df.empty:
                numeric_cols = df.select_dtypes(include='number').columns
                total_values = {}

                for col in numeric_cols:
                    if "price" in col.lower():
                        total_values[col] = df[col].mean()  # Take the average
                    else:
                        total_values[col] = df[col].sum()  # Take the sum

                # Ensure Grand Total is added only once
                if "Grand Total" not in df.iloc[:, 0].values:
                    grand_total_row = pd.DataFrame([[ "Grand Total" ] + [total_values.get(col, 0) for col in df.columns[1:]]], columns=df.columns)
                    df = pd.concat([df, grand_total_row], ignore_index=True)

            # Drop duplicate "Grand Total" rows, if they still exist
            df = df.drop_duplicates(subset=df.columns.tolist())


            outputdic[key] = df
            print(f"Query executed successfully for {hierby} | {row_field}.")
    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {hierby} {entity_type} | {row_field}: {db_error}")

    return outputdic


def process_dax_queries(hierarchy_levels, time_filter,area,row_list):
    with ThreadPoolExecutor(max_workers=8) as executor:
        for row_field in row_list:  # Process each row field separately
            dfs_results = {}  # Store results per row field
            futures = []
            ordered_keys=[]
            future_to_key={}
            for hierby, hier_values in hierarchy_levels:
                for value in hier_values:
                    future = executor.submit(execute_dax_query, hierby, time_filter, area , value, row_field=row_field)
                    key = f"{value}" 
                    ordered_keys.append(key)
                    futures.append(future)
                    future_to_key[future] = key  

            temp_results = {}
            for future in as_completed(future_to_key):
                key = future_to_key[future]
                try:
                    result = future.result()
                    temp_results.update(result)
                except Exception as e:
                    print(f"Failed to retrieve result for {key}: {e}")

            # Preserve original key order
            for key in ordered_keys:
                if key in temp_results:
                    dfs_results[key] = temp_results[key]
            # Save results separately for each row field
            if area == "RETAILER":
                filename = f"modified_retailer_{row_field.split('[')[-1][:-1]}.pkl"
            elif area == "CHANNEL":
                filename = f"modified_channels_{row_field.split('[')[-1][:-1]}.pkl"
            elif area == customareas:
                filename = f"modified_cust_{row_field.split('[')[-1][:-1]}.pkl"
            output_file = f"{path}\\{filename}"

            with open(output_file, "wb") as f:
                pd.to_pickle(dfs_results, f)

            print(f"Saved DataFrames for {row_field} to {output_file}.")


if "RETAILER" in areas:
    RET_list = []
    if len(regions_RET) != 0:
        RET_list.append('Market[Region]')
    if len(channels_RET) != 0:
        RET_list.append('Market[Channel]')
    if len(market_RET) != 0:
        RET_list.append('Market[Market]')
    process_dax_queries(hierarchy_levels, time_filter=True,area="RETAILER",row_list=RET_list)
    
if "CHANNEL" in areas:
    CHA_list = []
    if len(regions_CHAN) != 0:
       CHA_list.append('Market[Region]')
    if len(channels_CHAN) != 0:
      CHA_list.append('Market[Channel]')
    if len(market_CHAN) != 0:
      CHA_list.append('Market[Market]')
    process_dax_queries(hierarchy_levels, time_filter=True,area="CHANNEL",row_list=CHA_list)

if customareas in areas:
    CUST_list=[]
    if len(regions_CUST)!=0:
        CUST_list.append('Market[Region]')
    if len(channels_CUST)!=0:
        CUST_list.append('Market[Channel]')
    if len(market_CUST)!=0:
        CUST_list.append('Market[Market]')
    process_dax_queries(hierarchy_levels, time_filter=True,area=f"{customareas}",row_list=CUST_list)



Query executed successfully for Category | Market[Channel].
Query executed successfully for Segment | Market[Channel].
Query executed successfully for Sector | Market[Channel].
Query executed successfully for Sector | Market[Channel].
Query executed successfully for Segment | Market[Channel].
Query executed successfully for SubSegment | Market[Channel].
Query executed successfully for SubSegment | Market[Channel].
Query executed successfully for Segment | Market[Channel].
Query executed successfully for SubCategory | Market[Channel].
Query executed successfully for SubSegment | Market[Channel].
Query executed successfully for SubSegment | Market[Channel].
Query executed successfully for SubSegment | Market[Channel].
Query executed successfully for SubSegment | Market[Channel].
Query executed successfully for SubCategory | Market[Channel].
Query executed successfully for SubCategory | Market[Channel].
Query executed successfully for SubSegment | Market[Channel].
Query executed successfu

## National Extraction

In [31]:
def execute_dax_query(hierby, filter_p12m ,entity_type=None):
    outputdic = {}

    time_filter = """
        FILTER(
            VALUES('Time Logic'[Time Period]), 
            'Time Logic'[Time Period] = "P12M"
        )
    """ if filter_p12m else ""

    key = f"{entity_type}"
    filters = ", ".join(filter(None, [time_filter]))

    columns = [
        "Trade WoB %", "Trade WoB % DYA", "Channel Growth Contribution",
        "Value Sales", "Value Sales IYA", "Av Price/KG", "Channel Relative Price",
        "Value Share", "IYA Price/KG", "Value Share DYA"
    ]
    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)

    # Construct DAX query dynamically with a single row_field
    dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(Market, Market[Area]),
                {column_exprs}
            ),
            {filters},
            FILTER(Products, Products[{hierby}] = "{entity_type}"),
            FILTER('Scope', 'Scope'[Scope] = "{hierby}")
        )
    """

    
    with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
        cursor.execute(dax_query)
        columns = [desc[0] for desc in cursor.description]
        data = cursor.fetchall()

        df = pd.DataFrame(data, columns=columns)
        df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)         
        df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]  # Remove zero rows
        df.columns = df.columns.str.replace('Trade ', '').str.replace('Channel ','').str.strip()
        df = df.sort_values('WoB %', ascending=False)              
        if not df.empty:
            numeric_cols = df.select_dtypes(include='number').columns
            total_values = {}

            for col in numeric_cols:
                if "price" in col.lower():
                    total_values[col] = df[col].mean()  # Take the average
                else:
                    total_values[col] = df[col].sum()  # Take the sum

            # Ensure Grand Total is added only once
            if "Grand Total" not in df.iloc[:, 0].values:
                grand_total_row = pd.DataFrame([[ "Grand Total" ] + [total_values.get(col, 0) for col in df.columns[1:]]], columns=df.columns)
                df = pd.concat([df, grand_total_row], ignore_index=True)

        # Drop duplicate "Grand Total" rows, if they still exist
        df = df.drop_duplicates(subset=df.columns.tolist())


        outputdic[key] = df
        # print(f"Query executed successfully for {hierby} | {row_field}.")
    # print(f"Database error for {hierby} {entity_type} | {row_field}: {db_error}")

    return outputdic


def process_dax_queries(hierarchy_levels, time_filter):
    with ThreadPoolExecutor(max_workers=8) as executor:
        # Process each row field separately
        dfs_results = {}
        futures = []
        ordered_keys = []
        future_to_key = {}

        for hierby, hier_values in hierarchy_levels:
                for value in hier_values:
                    future = executor.submit(execute_dax_query, hierby, time_filter, value)
                    key = f"{value}" 
                    ordered_keys.append(key)
                    futures.append(future)
                    future_to_key[future] = key  

        temp_results = {}
        for future in as_completed(future_to_key):
            key = future_to_key[future]
            try:
                result = future.result()
                temp_results.update(result)
            except Exception as e:
                print(f"Failed to retrieve result for {key}: {e}")
        for key in ordered_keys:
                if key in temp_results:
                    dfs_results[key] = temp_results[key]       
        
        filename = "modified_national.pkl"

        output_file = f"{path}\\{filename}"

        with open(output_file, "wb") as f:
            pd.to_pickle(dfs_results, f)

process_dax_queries(hierarchy_levels, time_filter=True) 

In [32]:
def execute_dax_query(hierby, filter_p12m ,entity_type=None, manuf=None, brand=None):
    outputdic = {}

    time_filter = """
        FILTER(
            VALUES('Time Logic'[Time Period]), 
            'Time Logic'[Time Period] = "P12M"
        )
    """ if filter_p12m else ""
    manuf_filter = f'Products[{ManufOrTopC}]="{manuf}"' if manuf else ""
    brand_filter = f'Products[{BrandOrTopB}]="{brand}", Products[{ManufOrTopC}]="{client_manuf[0]}"' if brand else ""

    key = f"{entity_type} | {brand}" if brand else f"{entity_type} | {manuf}" if manuf else ""
    filters = ", ".join(filter(None, [manuf_filter, brand_filter, time_filter]))

    columns = [
        "Trade WoB %", "Trade WoB % DYA", "Channel Growth Contribution",
        "Value Sales", "Value Sales IYA", "Av Price/KG", "Channel Relative Price",
        "Value Share", "IYA Price/KG", "Value Share DYA"
    ]
    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)

    # Construct DAX query dynamically with a single row_field
    dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(Market, Market[Area]),
                {column_exprs}
            ),
            {filters},
            FILTER(Products, Products[{hierby}] = "{entity_type}"),
            FILTER('Scope', 'Scope'[Scope] = "{hierby}")
        )
    """

    
    with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
        cursor.execute(dax_query)
        columns = [desc[0] for desc in cursor.description]
        data = cursor.fetchall()

        df = pd.DataFrame(data, columns=columns)
        df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)         
        df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]  # Remove zero rows
        df.columns = df.columns.str.replace('Trade ', '').str.replace('Channel ','').str.strip()
        df = df.sort_values('WoB %', ascending=False)              
        if not df.empty:
            numeric_cols = df.select_dtypes(include='number').columns
            total_values = {}

            for col in numeric_cols:
                if "price" in col.lower():
                    total_values[col] = df[col].mean()  # Take the average
                else:
                    total_values[col] = df[col].sum()  # Take the sum

            # Ensure Grand Total is added only once
            if "Grand Total" not in df.iloc[:, 0].values:
                grand_total_row = pd.DataFrame([[ "Grand Total" ] + [total_values.get(col, 0) for col in df.columns[1:]]], columns=df.columns)
                df = pd.concat([df, grand_total_row], ignore_index=True)

        # Drop duplicate "Grand Total" rows, if they still exist
        df = df.drop_duplicates(subset=df.columns.tolist())


        outputdic[key] = df
        # print(f"Query executed successfully for {hierby} | {row_field}.")
    # print(f"Database error for {hierby} {entity_type} | {row_field}: {db_error}")

    return outputdic


def process_dax_queries(hierarchy_levels, time_filter, client_manuf=None, client_brands=None):
    with ThreadPoolExecutor(max_workers=8) as executor:
        # Process each row field separately
        dfs_results = {}
        futures = []
        ordered_keys = []
        future_to_key = {}

        for hierby, hier_values in hierarchy_levels:
            for value in hier_values:
                for manuf in client_manuf:
                    future = executor.submit(execute_dax_query, hierby, time_filter, value, manuf=manuf)
                    key = f"{hierby}:{value} | {manuf}" 
                    ordered_keys.append(key)
                    futures.append(future)
                    future_to_key[future] = key  
                for brand in client_brands:
                    future = executor.submit(execute_dax_query, hierby, time_filter, value, brand=brand)
                    key = f"{hierby}:{value} | {brand}" 
                    ordered_keys.append(key)
                    futures.append(future)
                    future_to_key[future] = key  

        for future in as_completed(futures):
            key = future_to_key[future]
            try:
                result = future.result()
                dfs_results.update(result)
            except Exception as e:
                print(f"Failed to retrieve result for {key}: {e}")
        if client_brands:
            filename= "modified_national_client.pkl" 

        output_file = f"{path}\\{filename}"

        with open(output_file, "wb") as f:
            pd.to_pickle(dfs_results, f)

process_dax_queries(hierarchy_levels, time_filter=True, client_manuf=client_manuf, client_brands=client_brands)  


## Retailers, Channels, Custom For CLIENTS Dataframes


In [33]:
def execute_dax_query(hierby, filter_p12m,area,entity_type=None,row_field=None, brand=None, manuf=None):
    outputdic = {}

    time_filter = """
        FILTER(
            VALUES('Time Logic'[Time Period]), 
            'Time Logic'[Time Period] = "P12M"
        )
    """ if filter_p12m else ""

    manuf_filter = f'Products[{ManufOrTopC}]="{manuf}"' if manuf else ""
    brand_filter = f'Products[{BrandOrTopB}]="{brand}", Products[{ManufOrTopC}]="{client_manuf[0]}"' if brand else ""

    key = f"{entity_type} | {brand}" if brand else f"{entity_type} | {manuf}" if manuf else ""
    filters = ", ".join(filter(None, [manuf_filter, brand_filter, time_filter]))

    columns = [
        "Trade WoB %", "Trade WoB % DYA", "Channel Growth Contribution",
        "Value Sales", "Value Sales IYA", "Av Price/KG", "Channel Relative Price",
        "Value Share", "IYA Price/KG", "Value Share DYA"
    ]
    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)

    # Construct the DAX query dynamically
    dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(Market, {row_field}),
                {column_exprs}
            ),
            TREATAS({years}, Calendar[Year]),
            {filters},
            FILTER(Products,Products[{hierby}] = "{entity_type}"),
            Filter(Market,Market[Area]="{area}"),
            FILTER('Scope', 'Scope'[Scope] = "{hierby}")
            
        )   
    """
    try:
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(dax_query)
            columns = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()

            df = pd.DataFrame(data, columns=columns)
            df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)         
            df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]  # Remove zero rows
            df.columns = df.columns.str.replace('Trade ', '').str.replace('Channel ','').str.strip()
            df = df.sort_values('Value Share', ascending=False)  
            df = df.sort_values('WoB %', ascending=False)  
            df = df.drop_duplicates()

            if not df.empty:
                numeric_cols = df.select_dtypes(include='number').columns
                total_values = {}

                for col in numeric_cols:
                    if "price" in col.lower():
                        total_values[col] = df[col].mean()  # Take the average
                    else:
                        total_values[col] = df[col].sum()  # Take the sum

                # Ensure Grand Total is added only once
                if "Grand Total" not in df.iloc[:, 0].values:
                    grand_total_row = pd.DataFrame([[ "Grand Total" ] + [total_values.get(col, 0) for col in df.columns[1:]]], columns=df.columns)
                    df = pd.concat([df, grand_total_row], ignore_index=True)

            # Drop duplicate "Grand Total" rows, if they still exist
            df = df.drop_duplicates(subset=df.columns.tolist())

            outputdic[key] = df
            print(f"Query executed successfully for {hierby} | {row_field}.")
    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {hierby} {entity_type} | {row_field}: {db_error}")

    return outputdic


def process_dax_queries(hierarchy_levels, time_filter,area,row_list, client_brands=None, client_manuf=None):
    with ThreadPoolExecutor(max_workers=8) as executor:
        for row_field in row_list:  # Process each row field separately
            dfs_results = {}
            futures = []
            ordered_keys=[]
            future_to_key={}
            for hierby, hier_values in hierarchy_levels:
                for value in hier_values:
                    if client_brands:
                        for brand in client_brands:
                            future = executor.submit(execute_dax_query, hierby, time_filter,area,value,row_field=row_field,brand=brand)
                            key = f"{value } | {brand}"
                            ordered_keys.append(key)
                            futures.append(future)
                            future_to_key[future] = key  
                        for manuf in client_manuf:
                            future = executor.submit(execute_dax_query, hierby, time_filter,area,value,row_field=row_field,manuf=manuf)
                            key = f"{value } | {manuf}"
                            ordered_keys.append(key)
                            futures.append(future)
                            future_to_key[future] = key 
                            
                temp_results = {}
                for future in as_completed(future_to_key):
                    key = future_to_key[future]
                    try:
                        result = future.result()
                        temp_results.update(result)
                    except Exception as e:
                        print(f"Failed to retrieve result for {key}: {e}")

                # Preserve original key order
                for key in ordered_keys:
                    if key in temp_results:
                        dfs_results[key] = temp_results[key]

            # Construct the output file name correctly
            if client_manuf:
                if area == "RETAILER":
                    filename = f"modified_retailer_{row_field.split('[')[-1][:-1]}_client.pkl"
                elif area == "CHANNEL":
                    filename = f"modified_channels_{row_field.split('[')[-1][:-1]}_client.pkl"
                elif area == customareas:
                    filename = f"modified_cust_{row_field.split('[')[-1][:-1]}_client.pkl"
            
            output_file = f"{path}\\{filename}"

            with open(output_file, "wb") as f:
                pd.to_pickle(dfs_results, f)

            print(f"All DataFrames saved to {output_file}.")
            
if "RETAILER" in areas:
    RET_list = []
    if len(regions_RET) != 0:
        RET_list.append('Market[Region]')
    if len(channels_RET) != 0:
        RET_list.append('Market[Channel]')
    if len(market_RET) != 0:
        RET_list.append('Market[Market]')
    process_dax_queries(hierarchy_levels, time_filter=True,area="RETAILER",row_list=RET_list,client_brands=client_brands,client_manuf=client_manuf)
if "CHANNEL" in areas:
    CHA_list = []
    if len(regions_CHAN) != 0:
       CHA_list.append('Market[Region]')
    if len(channels_CHAN) != 0:
      CHA_list.append('Market[Channel]')
    if len(market_CHAN) != 0:
      CHA_list.append('Market[Market]')
    process_dax_queries(hierarchy_levels, time_filter=True,area="CHANNEL",row_list=CHA_list,client_brands=client_brands,client_manuf=client_manuf)

if customareas in areas:
    CUST_list=[]
    if len(regions_CUST)!=0:
        CUST_list.append('Market[Region]')
    if len(channels_CUST)!=0:
        CUST_list.append('Market[Channel]')
    if len(market_CUST)!=0:
        CUST_list.append('Market[Market]')
    process_dax_queries(hierarchy_levels, time_filter=True,area=f"{customareas}",row_list=CUST_list,client_brands=client_brands,client_manuf=client_manuf)

# process_dax_queries(hierarchy_levels, time_filter=True,client_brands=client_brands,client_manuf=client_manuf)  # Brand-level

Query executed successfully for Category | Market[Channel].
Query executed successfully for Category | Market[Channel].
Query executed successfully for Category | Market[Channel].
Query executed successfully for Category | Market[Channel].
Query executed successfully for Sector | Market[Channel].
Query executed successfully for Sector | Market[Channel].
Query executed successfully for Sector | Market[Channel].
Query executed successfully for Sector | Market[Channel].
Query executed successfully for Sector | Market[Channel].
Query executed successfully for Sector | Market[Channel].
Query executed successfully for Sector | Market[Channel].
Query executed successfully for Sector | Market[Channel].
Query executed successfully for Segment | Market[Channel].
Query executed successfully for Segment | Market[Channel].
Query executed successfully for Segment | Market[Channel].
Query executed successfully for Segment | Market[Channel].
Query executed successfully for Segment | Market[Channel].
Q

### Momentum Analysis Dataframes

In [34]:
def execute_dax_query(hierby, area ,entity_type=None, row_field=None):
    outputdic = {}
    time_periods = '{"P3M", "P12M"}'  # Correct format for DAX


    key = f"{entity_type} | {area} | {row_field.split('[')[-1][:-1]}"

    columns = [
        "Value Sales","Value Share", "Value Share DYA"
    ]
    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)

    # Construct DAX query dynamically with a single row_field
    dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                CROSSJOIN(
                
                SUMMARIZE(Products, Products[{BrandOrTopB}]),  
                SUMMARIZE(Market, {row_field}),
                VALUES('Time Logic'[Time Period])
                ),

                {column_exprs}
            ),
            TREATAS({{"P3M", "P12M"}}, 'Time Logic'[Time Period]),
            FILTER(Products, Products[{hierby}] = "{entity_type}"),
            FILTER(Market, Market[Area] = "{area}"),
            FILTER('Scope', 'Scope'[Scope] = "{hierby}")
        )
    """

    try:
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(dax_query)
            columns = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()

            df = pd.DataFrame(data, columns=columns)
            df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)         
            df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]  # Remove zero rows
            df.columns = df.columns.str.replace('Trade ', '').str.replace('Channel ','').str.strip()
            df = df.drop_duplicates()
            outputdic[key] = df
            print(f"Query executed successfully for {hierby} | {row_field}.")
    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {hierby} {entity_type} | {row_field}: {db_error}")

    return outputdic


def process_dax_queries(hierarchy_levels,area,row_list):
    with ThreadPoolExecutor(max_workers=8) as executor:
        for row_field in row_list:  # Process each row field separately
            dfs_results = {}  # Store results per row field
            futures = []
            ordered_keys=[]
            future_to_key={}
            for hierby, hier_values in hierarchy_levels:
                for value in hier_values:
                    future = executor.submit(execute_dax_query, hierby, area , value, row_field=row_field)
                    key = f"{value} | {area} | {row_field.split('[')[-1][:-1]}"
                    ordered_keys.append(key)
                    futures.append(future)
                    future_to_key[future] = key 
            temp_results = {}
            for future in as_completed(future_to_key):
                key = future_to_key[future]
                try:
                    result = future.result()
                    temp_results.update(result)
                except Exception as e:
                    print(f"Failed to retrieve result for {key}: {e}")

            # Preserve original key order
            for key in ordered_keys:
                if key in temp_results:
                    dfs_results[key] = temp_results[key]
            # Save results separately for each row field
            if area == "RETAILER":
                filename = f"modified_momentum_retailer_{row_field.split('[')[-1][:-1]}.pkl"
            elif area == "CHANNEL":
                filename = f"modified_momentum_channels_{row_field.split('[')[-1][:-1]}.pkl"
            elif area == customareas:
                filename = f"modified_momentum_cust_{row_field.split('[')[-1][:-1]}.pkl"
            output_file = f"{path}\\{filename}"

            with open(output_file, "wb") as f:
                pd.to_pickle(dfs_results, f)

            print(f"Saved DataFrames for {row_field} to {output_file}.")


if "RETAILER" in areas:
    RET_list = []
    if len(regions_RET) != 0:
        RET_list.append('Market[Region]')
    if len(channels_RET) != 0:
        RET_list.append('Market[Channel]')
    if len(market_RET) != 0:
        RET_list.append('Market[Market]')
    process_dax_queries(hierarchy_levels, area="RETAILER",row_list=RET_list)
    
if "CHANNEL" in areas:
    CHA_list = []
    if len(regions_CHAN) != 0:
       CHA_list.append('Market[Region]')
    if len(channels_CHAN) != 0:
      CHA_list.append('Market[Channel]')
    if len(market_CHAN) != 0:
      CHA_list.append('Market[Market]')
    process_dax_queries(hierarchy_levels, area="CHANNEL",row_list=CHA_list)

if customareas in areas:
    CUST_list=[]
    if len(regions_CUST)!=0:
        CUST_list.append('Market[Region]')
    if len(channels_CUST)!=0:
        CUST_list.append('Market[Channel]')
    if len(market_CUST)!=0:
        CUST_list.append('Market[Market]')
    process_dax_queries(hierarchy_levels, area=f"{customareas}",row_list=CUST_list)



Query executed successfully for Category | Market[Channel].
Query executed successfully for Segment | Market[Channel].
Query executed successfully for Sector | Market[Channel].
Query executed successfully for SubSegment | Market[Channel].
Query executed successfully for Segment | Market[Channel].
Query executed successfully for Segment | Market[Channel].
Query executed successfully for SubSegment | Market[Channel].
Query executed successfully for Sector | Market[Channel].
Query executed successfully for SubSegment | Market[Channel].
Query executed successfully for SubSegment | Market[Channel].
Query executed successfully for SubSegment | Market[Channel].
Query executed successfully for SubSegment | Market[Channel].
Query executed successfully for SubSegment | Market[Channel].
Query executed successfully for SubCategory | Market[Channel].
Query executed successfully for SubCategory | Market[Channel].
Query executed successfully for SubCategory | Market[Channel].
Query executed successfu

### Momentum{"P3M", "P12M"} Dataframes

In [35]:
def execute_dax_query(entity_name, area):
    outputdic = {}
    time_periods = '{"P3M", "P12M"}'  # Correct format for DAX

    key = f"{categories[0]} | {entity_name}"
    columns = ["Value Sales", "Value Share","Value Share DYA"]
    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)

    dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                CROSSJOIN(
                    SUMMARIZE(Products, Products[{BrandOrTopB}]), 
                    SUMMARIZE(Products, Products[Sector]), 
                    VALUES('Time Logic'[Time Period])
                ),
                {column_exprs}
            ),
            TREATAS({time_periods}, 'Time Logic'[Time Period]),
            Products[Category] = "{categories[0]}",
            TREATAS({{"{entity_name}"}}, Market[{area}]),
            FILTER('Scope', 'Scope'[Scope] = "Category")
        )   
    """
    try:
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(dax_query)
            columns = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()

            df = pd.DataFrame(data, columns=columns)
            df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)         
            df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]  # Remove zero rows
            outputdic[key] = df
            
            print(f"Query executed successfully for {entity_name}.")
    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_name} in {area}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_name} in {area}: {e}")

    return outputdic

def process_dax_queries(entity_hierarchy):
    dfs_results = {}
    with ThreadPoolExecutor(max_workers=8) as executor:
        futures = []
        ordered_keys=[]
        future_to_key={}
        for area, entity_list in entity_hierarchy:
            for entity in entity_list:
                future = executor.submit(execute_dax_query, entity, area)
                key = f"{categories[0]} | {entity}"
                ordered_keys.append(key)
                futures.append(future)
                future_to_key[future] = key     
                
        temp_results = {}
        for future in as_completed(future_to_key):
            key = future_to_key[future]
            try:
                result = future.result()
                temp_results.update(result)
            except Exception as e:
                print(f"Failed to retrieve result for {key}: {e}")

        # Preserve original key order
        for key in ordered_keys:
            if key in temp_results:
                dfs_results[key] = temp_results[key]

        # Construct the output file name correctly
        filename = "momentum_dfs.pkl"
        output_file = f"{path}\\{filename}"
        with open(output_file, "wb") as f:
            pd.to_pickle(dfs_results, f)

        print(f"All DataFrames saved to {output_file}.")


process_dax_queries(entity_hierarchy)


Query executed successfully for Conv.
Query executed successfully for Food.
Query executed successfully for Walmart.
Query executed successfully for Sam's Corp.
Query executed successfully for NATIONAL.
Query executed successfully for Drug.
Query executed successfully for Bj's Corp.
All DataFrames saved to c:\Users\Ali Salem\Desktop\App_Update\Landscape\Landscape Datasets Test\momentum_dfs.pkl.


### Momentum hierarchy Dataframes

In [36]:
def execute_dax_query(area,row_field=None):
    outputdic = {}
    time_periods = '{"P3M", "P12M"}'  # Correct format for DAX

    key = f"{categories[0]} | {area} | {row_field.split('[')[-1][:-1]}"
    
    columns = ["Value Sales", "Value Share","Value Share DYA"]
    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)
    row_list = [
        f"SUMMARIZE(Products, Products[{BrandOrTopB}])"
    ]
    row_list.extend([
    f"SUMMARIZE(Market, {row_field})"
    ])
    if her_list:
        row_list.append(f"SUMMARIZE(Products, {', '.join(her_list)})")
    row_list.extend(["VALUES('Time Logic'[Time Period])"])

 
    dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                CROSSJOIN(
                    {", ".join(row_list)}

                ),
                {column_exprs}
            ),
            TREATAS({time_periods}, 'Time Logic'[Time Period]),
            Products[Category] = "{categories[0]}",
            FILTER(Market, Market[Area] = "{area}"),
            FILTER('Scope', 'Scope'[Scope] = "Category")
        )   
    """
    try:
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(dax_query)
            columns = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()

            df = pd.DataFrame(data, columns=columns)
            df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)         
            df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]  # Remove zero rows
            if not df.empty:
                numeric_cols = df.select_dtypes(include='number').columns
                total_values = {}

                for col in numeric_cols:
                    if "price" in col.lower():  # Check if the column name contains "price" (case insensitive)
                        total_values[col] = df[col].mean()  # Take the average
                    else:
                        total_values[col] = df[col].sum()  # Take the sum

                grand_total_row = pd.DataFrame([["Grand Total"] + [total_values.get(col, " ") for col in df.columns[1:]]], columns=df.columns)
                df = pd.concat([df, grand_total_row], ignore_index=True)
            outputdic[key] = df
            print(f"Query executed successfully for {area}.")
    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {row_field} in {area}: {db_error}")
    
    return outputdic


def process_dax_queries(area, row_list,her=None):
    with ThreadPoolExecutor(max_workers=8) as executor:
        for row_field in row_list:  # Process each row field separately
            dfs_results = {}
            futures = []        
            ordered_keys=[]
            future_to_key={}
            future = executor.submit(execute_dax_query, area, row_field=row_field)
            key = f"{categories[0]} | {area} | {row_field.split('[')[-1][:-1]}"
            ordered_keys.append(key)
            futures.append(future)
            future_to_key[future] = key       
            temp_results = {}
            for future in as_completed(future_to_key):
                key = future_to_key[future]
                try:
                    result = future.result()
                    temp_results.update(result)
                except Exception as e:
                    print(f"Failed to retrieve result for {key}: {e}")

            # Preserve original key order
            for key in ordered_keys:
                if key in temp_results:
                    dfs_results[key] = temp_results[key]

            if area == "RETAILER":
                filename = f"modified_momentum_{her}_retailer_{row_field.split('[')[-1][:-1]}.pkl"
            elif area == "CHANNEL":
                filename = f"modified_momentum_{her}_channels_{row_field.split('[')[-1][:-1]}.pkl"
            elif area == customareas:
                filename = f"modified_momentum_{her}_cust_{row_field.split('[')[-1][:-1]}.pkl"
            else:
                raise ValueError(f"Unexpected area: {area}. Cannot determine filename.")

            output_file = f"{path}\\{filename}"
            
            with open(output_file, "wb") as f:
                pd.to_pickle(dfs_results, f)

            print(f"All DataFrames saved to {output_file}.")
if any([segments, subsegments, subcategories]):
    if "RETAILER" in areas:
        RET_list=[]
        if regions_RET:
            RET_list.append('Market[Region]')
        if channels_RET:
            RET_list.append('Market[Channel]')
        if market_RET:
            RET_list.append('Market[Market]')
        if segments:
            her_list = []
            her_list.append("Products[Sector]")
            her_list.append("Products[Segment]")
            process_dax_queries(area="RETAILER", row_list=RET_list,her="seg")
        if subsegments:
            her_list = []         
            her_list.append('Products[Sector]')    
            her_list.append('Products[SubSegment]')  
            process_dax_queries(area="RETAILER", row_list=RET_list,her="subseg")
        if subcategories:   
            her_list = []      
            her_list.append('Products[Sector]')    
            her_list.append('Products[SubCategory]')     
            process_dax_queries(area="RETAILER", row_list=RET_list,her="subcat")

    if "CHANNEL" in areas:
        CHA_list=[]
        if len(regions_CHAN) != 0:
            CHA_list.append('Market[Region]')
        if len(channels_CHAN) != 0:
            CHA_list.append('Market[Channel]')
        if len(market_CHAN) != 0:
            CHA_list.append('Market[Market]')
        if segments:
            her_list = []
            her_list.append("Products[Sector]")
            her_list.append("Products[Segment]")
            process_dax_queries(area="CHANNEL",row_list=CHA_list,her="seg")
        if subsegments:
            her_list = []
            her_list.append("Products[Sector]")
            her_list.append("Products[SubSegment]")
            process_dax_queries(area="CHANNEL",row_list=CHA_list,her="subseg")
        if subcategories:
            her_list = []
            her_list.append("Products[Sector]")
            her_list.append("Products[SubCategory]")
            process_dax_queries(area="CHANNEL",row_list=CHA_list,her="subcat")        

    if customareas in areas:
        CUST_list=[]
        if len(regions_CUST)!=0:
            CUST_list.append('[Market].[Region]')
        if len(channels_CUST)!=0:
            CUST_list.append('[Market].[Channel]')
        if len(market_CUST)!=0:
            CUST_list.append('[Market].[Market]')
        if segments:
            her_list = []
            her_list.append("Products[Sector]")
            her_list.append("Products[Segment]")    
            process_dax_queries(area=f"{customareas}",row_list=CUST_list,her="seg")
        if subsegments:
            her_list = []
            her_list.append("Products[Sector]")
            her_list.append("Products[SubSegment]")    
            process_dax_queries(area=f"{customareas}",row_list=CUST_list,her="subseg")
        if subcategories:
            her_list = []
            her_list.append("Products[Sector]")
            her_list.append("Products[SubCategory]")    
            process_dax_queries(area=f"{customareas}",row_list=CUST_list,her="subcat")


Query executed successfully for RETAILER.
All DataFrames saved to c:\Users\Ali Salem\Desktop\App_Update\Landscape\Landscape Datasets Test\modified_momentum_seg_retailer_Channel.pkl.
Query executed successfully for RETAILER.
All DataFrames saved to c:\Users\Ali Salem\Desktop\App_Update\Landscape\Landscape Datasets Test\modified_momentum_subseg_retailer_Channel.pkl.
Query executed successfully for RETAILER.
All DataFrames saved to c:\Users\Ali Salem\Desktop\App_Update\Landscape\Landscape Datasets Test\modified_momentum_subcat_retailer_Channel.pkl.
Query executed successfully for CHANNEL.
All DataFrames saved to c:\Users\Ali Salem\Desktop\App_Update\Landscape\Landscape Datasets Test\modified_momentum_seg_channels_Region.pkl.
Query executed successfully for CHANNEL.
All DataFrames saved to c:\Users\Ali Salem\Desktop\App_Update\Landscape\Landscape Datasets Test\modified_momentum_subseg_channels_Region.pkl.
Query executed successfully for CHANNEL.
All DataFrames saved to c:\Users\Ali Salem\D

### Revenue PVM Dataframes

In [37]:
def execute_dax_query(BrandOrManuf,entity_name, area, hierby, filter_p12m,entity_type=None):
    outputdic = {}
    
    time_filter = """
        FILTER(
            VALUES('Time Logic'[Time Period]), 
            'Time Logic'[Time Period] = "P12M"
        )
    """ if filter_p12m else ""
    key = f"{entity_type} | {entity_name}" 
    filters = ", ".join(filter(None, [time_filter]))

    # Define columns dynamically
    columns = [
         "Revenue By PVM","Value Share"]
    

    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)


    dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                CROSSJOIN(
   
                SUMMARIZE(Products, Products[{BrandOrManuf}]), 
                SUMMARIZE(PVM, PVM[Revenue Group]) 
              ),  
                {column_exprs}
            ),
            {time_filter},
            FILTER(Products,Products[{hierby}] = "{entity_type}"),
            TREATAS({{"{entity_name}"}}, Market[{area}]),   
            FILTER('Scope', 'Scope'[Scope] = "{hierby}")
        )   
    """

    try:
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(dax_query)
            columns = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()

            df = pd.DataFrame(data, columns=columns)
            df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)         
            df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]  # Remove zero rows
            if not df.empty:
                numeric_cols = df.select_dtypes(include='number').columns
                total_values = {}

                for col in numeric_cols:
                    if "price" in col.lower():  # Check if the column name contains "price" (case insensitive)
                        total_values[col] = df[col].mean()  # Take the average
                    else:
                        total_values[col] = df[col].sum()  # Take the sum

                grand_total_row = pd.DataFrame([["Grand Total"] + [total_values.get(col, " ") for col in df.columns[1:]]], columns=df.columns)
                df = pd.concat([df, grand_total_row], ignore_index=True)

            outputdic[key] = df
            
            print(f"Query executed successfully for {entity_name}.")
    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_name} in {area}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_name} in {area}: {e}")

    return outputdic



def process_dax_queries(BrandOrManuf,entity_hierarchy, hierarchy_levels, time_filter):
    dfs_results = {}
    with ThreadPoolExecutor(max_workers=8) as executor:
        futures = []
        ordered_keys=[]
        future_to_key={}
        for hierby, hier_values in hierarchy_levels:
            for value in hier_values:
                for area, entity_list in entity_hierarchy:
                    for entity in entity_list:                    
                        future = executor.submit(execute_dax_query,BrandOrManuf, entity, area, hierby, time_filter,value)
                        key = f"{value} | {entity}"
                        ordered_keys.append(key)
                        futures.append(future)
                        future_to_key[future] = key     
        temp_results = {}
        for future in as_completed(future_to_key):
            key = future_to_key[future]
            try:
                result = future.result()
                temp_results.update(result)
            except Exception as e:
                print(f"Failed to retrieve result for {key}: {e}")

        # Preserve original key order
        for key in ordered_keys:
            if key in temp_results:
                dfs_results[key] = temp_results[key]

        # Construct the output file name correctly
        if BrandOrManuf==f'{BrandOrTopB}':
            filename = f"revenue_PVM_brand.pkl"
        else:
            filename = f"revenue_PVM_manuf.pkl"

        output_file = f"{path}\\{filename}"

        with open(output_file, "wb") as f:
            pd.to_pickle(dfs_results, f)

        print(f"All DataFrames saved to {output_file}.")


process_dax_queries(f"{BrandOrTopB}",entity_hierarchy,hierarchy_levels, time_filter=True)  # Brand-level
process_dax_queries(f"{ManufOrTopC}",entity_hierarchy,hierarchy_levels, time_filter=True)  # Brand-level


Query executed successfully for Bj's Corp.
Query executed successfully for Sam's Corp.
Query executed successfully for Conv.
Query executed successfully for NATIONAL.
Query executed successfully for Food.
Query executed successfully for Walmart.
Query executed successfully for Drug.
Query executed successfully for NATIONAL.
Query executed successfully for Sam's Corp.
Query executed successfully for Conv.
Query executed successfully for Drug.
Query executed successfully for Food.
Query executed successfully for Conv.
Query executed successfully for Bj's Corp.
Query executed successfully for NATIONAL.
Query executed successfully for Walmart.
Query executed successfully for Sam's Corp.
Query executed successfully for Food.
Query executed successfully for Drug.
Query executed successfully for Walmart.
Query executed successfully for Bj's Corp.
Query executed successfully for NATIONAL.
Query executed successfully for Food.
Query executed successfully for Conv.
Query executed successfully fo

In [38]:
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Script started at: {time.ctime(start_time)}")
print(f"Script ended at: {time.ctime(end_time)}")
print(f"Elapsed time: {elapsed_time:.2f} seconds")

Script started at: Wed Sep  3 14:58:34 2025
Script ended at: Wed Sep  3 15:17:42 2025
Elapsed time: 1147.99 seconds


In [39]:
1265.01 /60

21.0835

## Reading Pickle

In [40]:
loaded_data = {}
datasets_path =os.getcwd()+"\\Landscape Datasets Test\\"
datasets = os.listdir(datasets_path)
for d in datasets:
    with open(datasets_path+d, 'rb') as handle:
        globals()[d.split('.')[0]] = pd.read_pickle(handle)